In [2]:
# import
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
import pandas as pd

In [2]:
# recuperation des stopwords
# nltk.download('stopwords')
# stopwords_french = set(stopwords.words('french'))

[nltk_data] Downloading package stopwords to /home/cytech/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_csv('data.csv')
liste_tech = df[(df['typedictionnaire'] == 'tec') & (df['1'])]

In [4]:
print(liste_tech["1"])

1100             Management (ISO 50 001)
1101                Gestion des énergies
1102                        Organisation
1103                            Bâtiment
1104                         Bureautique
                      ...               
3806                  Gestión y pilotaje
3807                       Mantenimiento
3808                    Material rodante
3809                     Translado modal
3810    Indicadores y paneles de control
Name: 1, Length: 740, dtype: object


In [73]:
technologies = []

for phrase in liste_tech["1"]:
    technologies.append(str(phrase))

vectorizer = CountVectorizer()

X_train = [x for x in technologies if x not in stopwords_french]
X_train = vectorizer.fit_transform(X_train)

entrainement = X_train.toarray()

print("Vocabulaire:")
print(vectorizer.get_feature_names_out())

print("\nReprésentations BoW des documents:")
print(entrainement)

# new_sentence = "J'aimerais avoir une aide au management (ISO 50 001)"

new_sentence = "Comment faire pour réduire la consommation du groupe froid de mon compresseur d'air comprimé ?"

new_sentence_bow = vectorizer.transform([new_sentence])

print("\nReprésentation BoW de la phrase à tester:")

calc = new_sentence_bow.toarray()
print(calc)

result = []
for j in range(len(entrainement)):
    val = 1
    for i in range(len(entrainement[j])):
        if entrainement[j][i] == 1 and calc[0][i] == 0:
            val = 0
    if(val == 1):
        result.append(j)

tech_reliees = []
if(result != []):
    print("\nCas associés:")
    for x in result:
        tech_reliees.append(technologies[x])
else:
    print("Aucune reconnaissance")

Vocabulaire:
['001' '2d' '50' 'actif' 'adaptación' 'adjustment' 'administración' 'agua'
 'air' 'aire' 'aislamiento' 'ajustement' 'alimentaire' 'alimentaria'
 'almacenamiento' 'alumbrado' 'and' 'anti' 'aparatos' 'appareils'
 'appliances' 'appoint' 'armario' 'armoire' 'aspiration' 'assainissement'
 'audit' 'auditorías' 'aumentador' 'auxiliar' 'auxiliary' 'awareness'
 'aéraulique' 'bande' 'baterías' 'batterie' 'battery' 'besoin'
 'biocarburant' 'biocarburantes' 'biocarburants' 'biofuels' 'biogas'
 'biogaz' 'biogás' 'biologique' 'biomasa' 'biomass' 'biomasse' 'blower'
 'boiler' 'bomba' 'booster' 'booth' 'bord' 'boîtes' 'brûleur' 'buffer'
 'building' 'bureautique' 'burner' 'bâche' 'bâtiment' 'cabina' 'cabine'
 'cabinet' 'caldera' 'calderas' 'calefacción' 'calibreur' 'caliente'
 'calor' 'cambio' 'capacitor' 'capteur' 'captor' 'captores' 'captura'
 'capture' 'carbon' 'carpooling' 'cell' 'cellules' 'central' 'centrale'
 'chaleur' 'chambre' 'change' 'chargeur' 'chaude' 'chaudière' 'chauffage'
 

In [74]:
tech_reliees = set(tech_reliees)
print(tech_reliees)

{'Groupe froid', 'Froid', 'Air comprimé', 'Compresseur'}


In [28]:
num_tech = []

for x in tech_reliees:
    num_tech.append(df[(df['typedictionnaire'] == 'tec') & (df['1'] == x)]["codeappelobjet"])

Index([1100, 2373], dtype='int64')


In [35]:
solutions = []
df_sol = pd.read_csv('solution_techno.csv')
for x in num_tech:
    cles = x.keys()
    for k in cles:
        solutions.append(df_sol[(df_sol['codetechno'] == int(x[k]))]["numsolution"])

print(solutions)

[1    [428, 430, 442, 449, 901, 921, 950, 970, 974, ...
Name: numsolution, dtype: object, 1    [428, 430, 442, 449, 901, 921, 950, 970, 974, ...
Name: numsolution, dtype: object]


In [71]:
print(solutions)

[1    [428, 430, 442, 449, 901, 921, 950, 970, 974, ...
Name: numsolution, dtype: object, 1    [428, 430, 442, 449, 901, 921, 950, 970, 974, ...
Name: numsolution, dtype: object]


In [70]:
for x in solutions:
    temp = x[1][1:-1]
    liste_solution = temp.split(",")
    for j in liste_solution:
        sol_temp = df[(df['typedictionnaire'] == 'sol') & (df['codeappelobjet'] == int(j)) & (df["codelangue"] == 2)]
        i = 1
        fin = False
        while(not fin):
            for x in sol_temp[str(i)]:
                if pd.isna(x):
                    fin = True
                else:
                    print(x)
                    i += 1

Politique environnementale
Devant les exigences réglementaires de programmes européens et internationaux de lutte pour la réduction de la consommation énergétique et de diminution des émissions de gaz à effet de serre, il est devenu indispensable pour les entreprises et industries de développer une politique environnementale. Celle-ci doit comporter une liste d'objectifs montrant la volonté de la direction de s'engager pour le respect de l'environnement. La direction doit informer le personnel de ses objectifs et les mettre à disposition du public.
Le document signé par la direction, faisant état de la politique environnementale mise en oeuvre, concrétise l'implication responsable de l’entreprise dans la poursuite d’objectifs environnementaux.
Création d'un poste de responsable énergie
Le choix d'un responsable énergie est important, pour que le site avance sur les questions d'amélioration énergétique du site. Le responsable énergie devra assurer le suivi des consommations et des coûts

In [5]:
df = pd.read_csv('solution_rex.csv')
val = 1
solution = df["numsolution"]
moy = 0
nb = 1
val_temp = solution[0]
for i in range(1 ,len(solution)):
    if(val_temp == solution[i]):
        val += 1
    else:
        moy += val
        val = 1
        nb += 1
        val_temp = solution[i]
print(moy / nb)

4.395256916996048


[2] GJ = 2,7778 x 10-4 [3] GWh = 1000 [12] MWh = 1000000 [5] kWh = 1000000000 [37] W = 1,0551 [11] MMBtu
[14] tep = 42 [2]
[15] kW = [30] kVA

[13] tCO2e
[24] €/MWh
[31] kVAr ? nécessite Power Factor
[32] Wc (puissance maximale d'un panneau solaire)
[33] €
[34] % utile ?
[36] = tCO2/tep = [13] / (42 * [2])
[43] Ah (on a besoin de V pour en faire qqch)
[44] kWh/u (je sais pas ce que représente u)
A voir [6] kWh cumac
Pareil avec [47] MWh cumac
[50] kWh/tonne
[51] tonne /an
[52] kWh cumac / kW
[55] kWh cumac / W
[56] MWh/an
[58] h
[59] EUR
[60] month

In [8]:
PRIX_kWh = 0.2516

def calcul_cout(num_solution):
    df_soluce = pd.read_csv('solution_rex.csv')
    liste_soluce = df_soluce[(df_soluce['numsolution'] == int(num_solution))]
    consommation = 0
    nb_conso = 0
    for i in range(len(liste_soluce["energie_gain"])):
        if(not pd.isna(liste_soluce["energie_gain"].iloc[i])):
            if(liste_soluce["code_unite_energie"].iloc[i] == 2):
                consommation += 2.7778 * liste_soluce["energie_gain"].iloc[i] * 1000000 / 10000
            elif(liste_soluce["code_unite_energie"].iloc[i] == 3):
                consommation += liste_soluce["energie_gain"].iloc[i] * 1000000
            elif(liste_soluce["code_unite_energie"].iloc[i] == 12):
                consommation += liste_soluce["energie_gain"].iloc[i] * 1000
            elif(liste_soluce["code_unite_energie"].iloc[i] == 5):
                consommation += liste_soluce["energie_gain"].iloc[i]
            elif(liste_soluce["code_unite_energie"].iloc[i] == 37):
                consommation += liste_soluce["energie_gain"].iloc[i] / 1000
            elif(liste_soluce["code_unite_energie"].iloc[i] == 11):
                consommation += liste_soluce["energie_gain"].iloc[i] / 1.0551 / 1000
            elif(liste_soluce["code_unite_energie"].iloc[i] == 14):
                consommation += 42 * liste_soluce["energie_gain"].iloc[i] / (2.7778 * 10000) * 1000000
            else:
                print(liste_soluce["code_unite_energie"][i])
            nb_conso += 1
    if(nb_conso == 0):
        print("Pas de gain de conso")
    else:
        energie = consommation / nb_conso
        print(f"Gain d'énergie moyen: {energie} kWh")
        print(f"Soit {energie * PRIX_kWh} €")

    cout = 0
    nb_cout = 0
    df_monnaie = pd.read_csv('monnaie.csv')
    for i in range(len(liste_soluce["gain"])):
        if(not pd.isna(liste_soluce["gain"].iloc[i])):
            for j in range(len(df_monnaie["codemonnaie"])):
                if(liste_soluce["code_monnaie_gain"].iloc[i] == df_monnaie["codemonnaie"].iloc[j]):
                    cout += df_monnaie["rate"].iloc[j] * liste_soluce["gain"].iloc[i]
                    nb_cout += 1
    if(nb_cout == 0):
        print("Pas de gain de cout")
    else:
        print(f"Gain de cout moyen: {cout / nb_cout} €")

    co2 = 0
    nb_co2 = 0
    for i in range(len(liste_soluce["ges_gain"])):
        if(not pd.isna(liste_soluce["ges_gain"].iloc[i])):
            co2 += liste_soluce["ges_gain"].iloc[i]
            nb_co2 += 1
    if(nb_co2 == 0):
        print("Pas de gain de CO2")
    else:
        print(f"Gain de CO2 moyen: {co2 / nb_co2} tonnes")


calcul_cout(17)

Gain d'énergie: 436626.3172211165 kWh
Soit 109855.18141283291 €
Gain de cout: 704773.3333333334 €
Gain de CO2: 180.0 tonnes


In [13]:
import numpy as np
from statistics import mean

num_solution = 17

df_soluce = pd.read_csv('solution_rex.csv')
liste_soluce = df_soluce[(df_soluce['numsolution'] == int(num_solution))]
consommation = []
for i in range(len(liste_soluce["energie_gain"])):
    if(not pd.isna(liste_soluce["energie_gain"].iloc[i])):
        if(liste_soluce["code_unite_energie"].iloc[i] == 2):
            consommation.append(2.7778 * liste_soluce["energie_gain"].iloc[i] * 1000000 / 10000)
        elif(liste_soluce["code_unite_energie"].iloc[i] == 3):
            consommation.append(liste_soluce["energie_gain"].iloc[i] * 1000000)
        elif(liste_soluce["code_unite_energie"].iloc[i] == 12):
            consommation.append(liste_soluce["energie_gain"].iloc[i] * 1000)
        elif(liste_soluce["code_unite_energie"].iloc[i] == 5):
            consommation.append(liste_soluce["energie_gain"].iloc[i])
        elif(liste_soluce["code_unite_energie"].iloc[i] == 37):
            consommation.append(liste_soluce["energie_gain"].iloc[i] / 1000)
        elif(liste_soluce["code_unite_energie"].iloc[i] == 11):
            consommation.append(liste_soluce["energie_gain"].iloc[i] / 1.0551 / 1000)
        elif(liste_soluce["code_unite_energie"].iloc[i] == 14):
            consommation.append(42 * liste_soluce["energie_gain"].iloc[i] / (2.7778 * 10000) * 1000000)
        else:
            print(liste_soluce["code_unite_energie"][i])

# Données d'exemple
donnees = consommation  # Remplacez cela par vos propres données

# Calculer les logarithmes des données
log_donnees = np.log(donnees)

# Calculer la moyenne des logarithmes
moyenne_logarithmique = np.mean(log_donnees)

# Convertir la moyenne logarithmique en valeur normale si nécessaire
moyenne_valeurs = mean(donnees)

print("Moyenne logarithmique:", np.exp(moyenne_logarithmique))
print("Moyenne des valeurs originales:", moyenne_valeurs)

Moyenne logarithmique: 7.786123590979121
Moyenne des valeurs originales: 436626.3172211165
